In [ ]:
import pandas as pd 
from os import listdir
from os import path
from os import walk

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [ ]:
# Testing vaderSentiment
def vader_analyzer(sentence, stars):
    snt = vader.polarity_scores(sentence)
    if snt['compound'] >= 0.05: 
        if stars < 3:
            return f"Negative, {snt['compound']}"
        return f"Positive, {snt['compound']}"
    elif snt['compound'] <= - 0.05: 
        if stars > 3:
            return f"Positive, {snt['compound']}"
        return f"Negative, {snt['compound']}"
    else: 
        return f"Neutral, {snt['compound']}"


In [ ]:
from textblob import TextBlob

In [ ]:
# Testing TextBlob
def text_blob(sentence, stars):
    snt = TextBlob(sentence).sentiment
    if snt[0] > 0:
        # Check with number of stars
        if stars < 3 and snt[0] < 0.15:
            return f"Negative, {snt[0]:.4f}"
        return f"Positive, {snt[0]:.4f}"
    elif snt[0] < 0:
        # Check with number of stars
        if stars > 3 and snt[0] > -0.15: 
            return f"Positive, {snt[0]:.4f}" 
        return f"Negative, {snt[0]:.4f}"
    else:
        return f"Neutral, {snt[0]:.4f}"

In [ ]:
# Method to test one review file  
def test_review():
    test_files = {"Arts and Humanities": "Creative Writing: The Craft of Character- Wesleyan University-reviews.csv",
                  "Data Science": "Big Data Modeling and Management Systems-University of California San Diego-reviews.csv",
                  "Business": "Developing An Entrepreneurial Mindset: First Step Towards Success-Michigan State University-reviews.csv",
                  "Social Sciences": "Public Policy Challenges of the 21st Century-University of Virginia-reviews.csv"
                 }
    rev_df = pd.read_csv(test_files["Data Science"])
    revs = rev_df["Review Text"]
    
    # Create empty list and then append sentiment values, its more efficient than adding one by one to a series
    vadersent_list = []
    textbsent_list = []
    for i in range(len(revs)):
        print(f"Review is: '{revs.loc[i]}'")
        vadersent = vader_analyzer(str(revs.loc[i]), int(rev_df['Number of Stars'].loc[i]))
        textbsent = text_blob(revs.loc[i], int(rev_df['Number of Stars'].loc[i]))
        print(f"Stars are: {rev_df['Number of Stars'].loc[i]}, \nVaderSentiment: {vadersent} \nTextBlob Sentiment: {textbsent}")
        vadersent_list.append(vadersent)
        textbsent_list.append(textbsent)
        print()
    
    vadersent_cols = pd.Series(vadersent_list)
    textbsent_cols = pd.Series(textbsent_list)
    
    rev_df["VaderSentiment"] = vadersent_cols
    rev_df["TextBlobSentiment"] = textbsent_cols
    
    #print(rev_df)
    rev_df.to_csv("Data_Science_Test.csv", index=False)
    
test_review()

In [ ]:
# Main that calls all the analyzers to test
def main(): 
    dir_path = "~/Reviews/"
    
    # Read in CSV files, iterate through all folders
    for subdir, dirs, files in walk(dir_path):
        for file in files:
            filepath = path.join(subdir, file)
            if filepath.endswith(".csv"):
                print("Analyzing: " + filepath)
                rev_df = pd.read_csv(filepath)
                revs = rev_df["Review Text"]

                # Create empty list and then append sentiment values, its more efficient than adding one by one to a series
                vadersent_list = []
                # textbsent_list = []
                for i in range(len(revs)):
                    # print(f"Review is: '{revs.loc[i]}'")
                    vadersent = vader_analyzer(str(revs.loc[i]), int(rev_df['Number of Stars'].loc[i]))
                    # textbsent = text_blob(revs.loc[i], int(rev_df['Number of Stars'].loc[i]))
                    # print(f"Stars are: {rev_df['Number of Stars'].loc[i]}, \nVaderSentiment: {vadersent}")
                    vadersent_list.append(vadersent)
                    # textbsent_list.append(textbsent)
                    # print()

                vadersent_cols = pd.Series(vadersent_list)
                # textbsent_cols = pd.Series(textbsent_list)

                rev_df["VaderSentiment"] = vadersent_cols
                # rev_df["TextBlobSentiment"] = textbsent_cols

                # print(rev_df)
                rev_df.to_csv(filepath, index=False)

In [ ]:
main()